In [1]:
import pandas as pd

In [2]:
#=pd.load_csv(r'HTNcleanedl.csv')
dataset = pd.read_csv("DiabetesCleaned.csv", sep=None,engine='python')
dataset

,age,mets_achieved,resting_systolic,resting_diastolic,percent_hr_achieved,hyperlipid,famhx,black,newdm10
0,61.051334,5.0,132,84,0.805031,No,No,0,1
1,58.888432,5.0,138,80,0.937888,No,Yes,1,1
2,44.646133,10.1,100,74,0.954286,Yes,Yes,0,1
3,67.548256,7.0,150,90,0.842105,Yes,No,0,1
4,42.967831,7.0,110,80,0.903955,No,Yes,0,0
...,...,...,...,...,...,...,...,...,...
16771,40.626968,10.1,110,70,0.921788,No,No,0,0
16772,52.273785,10.0,134,72,0.958333,Yes,No,0,1
16773,43.225189,10.1,136,84,0.988701,No,Yes,0,0
16774,37.681042,10.1,98,82,0.939560,No,No,0,0


In [3]:
#data = dataset[[dataset.columns.values.tolist().remove('newdm10')]]
feature_names = dataset.columns.values.tolist()
feature_names.remove('newdm10')
feature_names

['age',
 'mets_achieved',
 'resting_systolic',
 'resting_diastolic',
 'percent_hr_achieved',
 'hyperlipid',
 'famhx',
 'black']

In [4]:
x = dataset.to_numpy()
x

array([[61.051334, 5.0, 132, ..., 'No', 0, 1],
       [58.888431999999995, 5.0, 138, ..., 'Yes', 1, 1],
       [44.646133, 10.1, 100, ..., 'Yes', 0, 1],
       ...,
       [43.225189, 10.1, 136, ..., 'Yes', 0, 0],
       [37.681042, 10.1, 98, ..., 'No', 0, 0],
       [46.551678, 7.0, 146, ..., 'Yes', 0, 0]], dtype=object)

In [5]:
def load_in_sklearn_format(filename,target_name):
    from bunch import Bunch
    import pandas as pd
    dataset = pd.read_csv(filename, sep=None,engine='python')
    dataset = pd.read_csv(filename, sep=None,engine='python')
    object_columns = dataset.select_dtypes(include='object')
    for column in object_columns :
        dataset[column] = dataset[column].astype('category')
    dataset=pd.get_dummies(dataset)
    feature_names = dataset.columns.values.tolist()
    feature_names.remove(target_name)
    data = dataset[feature_names].to_numpy()
    target = dataset[target_name].to_numpy()
    
    
    return Bunch(data=data, target=target, feature_names = feature_names, target_names = target_name)


In [6]:
x=load_in_sklearn_format('DiabetesCleaned.csv','newdm10')

In [7]:
data=x.data

In [8]:
data = x.data
feature_names = x.feature_names
y = x.target

In [9]:
from int_met import *

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
model = RandomForestClassifier()
#model=DecisionTreeClassifier()


In [11]:
train_data = data[1:300,:]
train_labels=y[1:300]
test_data = data[301:505,:]
test_labels=y[301:505]

In [12]:
model = model.fit(train_data,y=train_labels)

In [13]:
from sklearn.metrics import f1_score
y_pred = model.predict(test_data)
error = f1_score(test_labels,y_pred) 
error
y_pred

array([0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1])

In [14]:
#necessary imports
from pyBreakDown.explainer import Explainer
from pyBreakDown.explanation import Explanation

In [18]:
def run_PyBreakDown(model,data,observations,colnames):
    explanations=[]
    y=[]
    from pyBreakDown.explainer import Explainer
    from pyBreakDown.explanation import Explanation
    from math import exp as e
    exp = Explainer(clf=model, data=data, colnames=feature_names)
    if observations.ndim==1:
        explanation = exp.explain(observation=observations,direction="up")
        pred=explanation._final_prediction
        pred=round((e(pred)/(1+e(pred))))
        y.append(pred)
        explanations.append(explanation)
    else:
        for i in range(0,len(observations)):
            observation = observations[i]
            explanation = exp.explain(observation=observation,direction="up")
            pred=explanation._final_prediction
            pred=round((e(pred)/(1+e(pred))))
            y.append(pred)
            explanations.append(explanation)
    return explanations,y

In [19]:
def get_Coefs(explanations):
    coefs=[]
    for explanation in explanations:
        coef=[]
        for feature in explanation._attributes:
            coef.append(feature.contribution)
        coefs.append(coef)
    return coefs

In [20]:
explanations1,y1 = run_PyBreakDown(model=model,data=train_data,observations=data[500:569,:],colnames=feature_names)
explanations2,y2 = run_PyBreakDown(model=model,data=train_data,observations=data[500:569,:],colnames=feature_names)

In [21]:
coefs1 = get_Coefs(explanations1)
coefs2 = get_Coefs(explanations2)
y_test = y[500:569]
X_test = data[500:569,:]

In [22]:
calc_identity(coefs1,coefs2)

(0.0, 69, 69)

In [23]:
calc_separability(X_test, coefs1)

(69, 4, 94.20289855072464)

In [24]:
calc_compactness(coefs1)

0.4690382081686429

In [25]:
def calc_similarity(coefs1,X_test):
    from sklearn.preprocessing import normalize
    from sklearn.cluster import DBSCAN
    from sklearn.metrics.pairwise import euclidean_distances
    from sklearn import preprocessing 
    import numpy as np
    X_test = preprocessing.normalize(X_test, norm='l2')
    coefs1 = preprocessing.normalize(coefs1, norm='l2')
    clustering = DBSCAN(eps=0.02,min_samples=2).fit(X_test)
    cluster_labels = clustering.labels_
    unique_labels=np.unique(cluster_labels)
    mean_distances=[]
    for cluster in unique_labels:
        indexes_of_cluster=np.where(cluster_labels == cluster)[0]
        exps_of_cluster = [coefs1[i] for i in indexes_of_cluster] 
        mean_distances.append(np.average(euclidean_distances(exps_of_cluster)))
        
    return np.average(mean_distances)
        
calc_similarity(coefs1,X_test)

Cluster -1 indexes_of_cluster [ 0  1  2  5  6  7  8  9 10 11 12 13 14 16 18 19 20 21 23 24 25 26 28 29
 30 31 34 35 36 38 39 41 42 44 46 48 51 53 54 55 56 57 58 61 62 63 64 65
 66 68]
Cluster 0 indexes_of_cluster [ 3 17]
Cluster 1 indexes_of_cluster [ 4 47]
Cluster 2 indexes_of_cluster [15 32]
Cluster 3 indexes_of_cluster [22 27 40]
Cluster 4 indexes_of_cluster [33 37]
Cluster 5 indexes_of_cluster [43 60]
Cluster 6 indexes_of_cluster [45 67]
Cluster 7 indexes_of_cluster [49 50]
Cluster 8 indexes_of_cluster [52 59]


0.5406993210348141